In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks

### Callbacks as GUI events

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description='Click Me')

In [4]:
w

Button(description='Click Me', style=ButtonStyle())

In [5]:
def f(o): print('hello hello')

In [6]:
w.on_click(f)

**Q:** `on_click` is not working

### Creating your own callback

In [7]:
from time import sleep

In [8]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [9]:
slow_calculation()

30

In [10]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res

In [11]:
def show_progress(epoch): print(f'We have finished epoch {epoch}')

In [12]:
slow_calculation(show_progress)

We have finished epoch 0
We have finished epoch 1
We have finished epoch 2
We have finished epoch 3
We have finished epoch 4


30

### Lambdas and partials

In [13]:
slow_calculation(lambda o: print(f'We have finished epoch {o}'))

We have finished epoch 0
We have finished epoch 1
We have finished epoch 2
We have finished epoch 3
We have finished epoch 4


30

In [14]:
def show_progress(salutation, epoch): print(f'{salutation}, we have finished epoch {epoch}')

In [15]:
slow_calculation(lambda o: show_progress('Brother', o))

Brother, we have finished epoch 0
Brother, we have finished epoch 1
Brother, we have finished epoch 2
Brother, we have finished epoch 3
Brother, we have finished epoch 4


30

In [16]:
def make_show_progress(salutation):
    def _inner(epoch): print(f'{salutation}, we have finished epoch {epoch}')
    return _inner

In [17]:
slow_calculation(make_show_progress('Brah'))

Brah, we have finished epoch 0
Brah, we have finished epoch 1
Brah, we have finished epoch 2
Brah, we have finished epoch 3
Brah, we have finished epoch 4


30

In [18]:
from functools import partial

In [19]:
slow_calculation(partial(show_progress, 'Bruh'))

Bruh, we have finished epoch 0
Bruh, we have finished epoch 1
Bruh, we have finished epoch 2
Bruh, we have finished epoch 3
Bruh, we have finished epoch 4


30

In [20]:
f2 = partial(show_progress, 'Bruh')

### Callbacks as callable classes

In [21]:
class ProgressShowingCallback():
    def __init__(self, salutation='Hello'): self.salutation = salutation
    def __call__(self, epoch): print(f'{self.salutation}, we have finished epoch {epoch}')

Note Umer: Unlike cb functions, cb classes allow to store state

In [22]:
cb = ProgressShowingCallback('Bruh')

In [23]:
slow_calculation(cb)

Bruh, we have finished epoch 0
Bruh, we have finished epoch 1
Bruh, we have finished epoch 2
Bruh, we have finished epoch 3
Bruh, we have finished epoch 4


30

### Multiple callback funcs; `*args` and `**kwargs`

In [24]:
def f(*a, **b): print(f'args: {a}, kwargs: {b}')

In [25]:
f(3, 'a', things1='hello')

args: (3, 'a'), kwargs: {'things1': 'hello'}


In [26]:
def g(a,b,c=0): print(a,b,c)

In [27]:
args = [1,2]
kwargs = {'c':3}
g(*args,**kwargs)

1 2 3


In [28]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

In [29]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f'About to start')
    def after_calc (self, *args, **kwargs): print(f'Done step')

In [30]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [31]:
class PrintStepCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f'About to start {epoch}')
    def after_calc (self, epoch, val, **kwargs): print(f'After {epoch}: {val}')

In [32]:
slow_calculation(PrintStepCallback())

About to start 0
After 0: 0
About to start 1
After 1: 1
About to start 2
After 2: 5
About to start 3
After 3: 14
About to start 4
After 4: 30


30

### Modifying behavior

In [33]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, 'before_calc'): cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb, 'after_calc'):
            if cb.after_calc(i, res):
                print('stopping early')
                break
    return res

In [34]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f'After {epoch}: {val}')
        if val>10: return True

In [35]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

Note Umer: Following code simplifies the apparatures which allows callbacks

In [36]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res=cb,0
        
    def callback(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb,cb_name,None)
        if cb: return cb(self, *args)
    
    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(i)
            if self.callback('after_calc', i):
                print('stoppig early')
                break

In [37]:
class ModyfingCallback():
    def after_calc(self, calc, epoch):
        print(f'After {epoch}: {calc.res}')
        if calc.res>10: return True
        if calc.res<3: calc.res *= 2

In [38]:
calculator = SlowCalculator(ModyfingCallback())

In [39]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stoppig early


15

## `__dunder__` stuff

In [40]:
# Note Umer: This is actually not an 'adder' (ie a binary op),
# but rather a number, which when added to a number of same kind
# is sloppy
class SloppyAdder():
    def __init__(self,o): self.o=o
    def __add__(self,b): return SloppyAdder(self.o+b.o+0.01)
    def __repr__(self): return str(self.o)

In [41]:
a,b = SloppyAdder(1), SloppyAdder(2)
a+b

3.01

Special methods I should probably know about:
- `__getitem__` - [bla]
- `__getattr__` - .bla
- `__setitem__` - .bla = blub
- `__del__`
- `__init__`
- `__new__` - used to create a new instance of a class
- `__enter__` - when entering context manager
- `__exit__` - when leaving context manager
- `__len__`
- `__repr__` - 'official string representation'
- `__str__` - 'informat/nice string representation'

## `__getattr__` and `getattr`

In [42]:
class A: a,b=1,2

In [43]:
a = A()

In [44]:
a.b

2

In [45]:
getattr(a, 'b')

2

In [46]:
getattr(a, 'b' if random.random()>0.5 else 'a')

1

In [47]:
class B:
    a,b=1,2
    def __getattr__(self,k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello from non-private {k}'

In [48]:
b = B()
b.a, b.b

(1, 2)

In [49]:
b.fooooo

'Hello from non-private fooooo'